In [ ]:
# csce 487 intro to ML
# assignment 3 - Naive Bays Classifier & Logistic Regression
# collaborators : Changsu Kim, Jesse Reyes Cortes


#################             NAIVE BAYES CLASSIFIER             #################

# part A: model code (design multinomial naive bayes)


In [1]:
# part B: exploratory data analysis
# B.2 read in SMSSpamCollection.csv as DataFrame


In [ ]:
# B.3 summerization of data in terms of mean, standard deviation, and quartiles


In [ ]:
# B.4 generate plot to display class distribution


In [ ]:
# part C: Feature Extraction
# C.5 normalize text


In [ ]:
# C.6 generate word clouds for both spam and ham emails (NLTK lib)


In [ ]:
# C.7 remove stop words & convert to numerical feature vectors


In [ ]:
# C.8 create data 


In [ ]:
# C.9 shuffle rows of data 


In [ ]:
# C.10 partition data (80% train - 20% test)


In [ ]:
# part D: Model Evaluation
# D.11 model selection via hyper-parameter tuning


In [ ]:
# D.12 generate ROC and AUC (bonus)


In [ ]:
# D.13 evaluate model on test day (Precision, Recall, F1 score, Confusion matrix, Accuracy)


In [ ]:
# D.14 multivariate bernoulli naive bayes model (bonus)


In [ ]:
#################      LOGISTIC REGRESSION: MULTI-CLASS CLASSIFICATION      #################
# part A: model code
# A.15 design softmax  regression classifier


In [ ]:
# A.16 compute softmax score 


In [ ]:
# A.17 implement function to compute the cross-entropy loss


In [ ]:
# A.18 implement a softmax_regression model class


In [ ]:
# B.19 read iris data (use sklearn.datasets.load_iris)


In [ ]:
# B.20 summerize variables in terms of mean, standard deviation, and quartiles (use technique from 2nd recitation)


In [ ]:
# B.21 shuffle rows of data


In [ ]:
# B.22 generate plots (seaborn)


In [ ]:
# B.23 scale the features


In [ ]:
# B.24 partion data


In [ ]:
# part C: Model Evaluation
# C.25 model selection via hyper-parameter tuning


In [ ]:
# C.26 evaluate model on test data & report accuracy & confusion matrix


In [ ]:
# C.27 implement early stop in "fit" method for softmax regression model (bonus)


In [ ]:
# C.28 implement stochastic gradient descent logistic regression algorithm (bonus)